In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import math
import matplotlib
from scipy import ndimage, misc

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./tmp/data/t10k-images-idx3-ubyte.gz
Extracting ./tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
class STN(object):
    def __init__(self, sess, batch_size, epochs):
        self.input_size = 28
        self.output_size = 28
        self.batch_size = batch_size
        self.epochs = epochs
        self.sess = sess
        
        self.initial = np.array([[1., 0., 0.], [0., 1., 0.]], dtype=np.float32)
        self.initial = self.initial.flatten()
        
        self.initial = np.expand_dims(self.initial, axis=0)
        

        self.X = tf.placeholder(dtype=tf.float32, shape=[None, self.input_size, self.input_size, 1])
        self.Y = tf.placeholder(dtype=tf.float32, shape=[None, self.output_size, self.output_size, 1])
        #self.Output = tf.placeholder(dtype = tf.float32, shape=[None, self.output_size, self.output_size, 1])
        
        self.W1 = tf.Variable(tf.random_normal(shape=[3, 3, 1, 32], stddev=0.01))
        self.W2 = tf.Variable(tf.random_normal(shape=[3, 3, 32, 64], stddev=0.01))
        self.W3 = tf.Variable(tf.random_normal(shape=[3, 3, 64, 128], stddev=0.01))
        self.W4 = tf.Variable(tf.random_normal(shape=[3, 3, 128, 256], stddev=0.01))
        self.W5 = tf.Variable(tf.random_normal(shape=[3, 3, 256, 128], stddev=0.01))
        self.W6 = tf.Variable(tf.random_normal(shape=[3, 3, 128, 64], stddev=0.01))
        self.W7 = tf.Variable(tf.random_normal(shape=[7*7*64, 32], stddev=0.01))
        self.W8 = tf.Variable(tf.zeros(shape=[32, 6]))
        self.b8 = tf.Variable(self.initial)
        
        
        self.model()
        #self.Get_Pixel_Value()
    
    def Localisation_Net(self, img):
        #img -> [batch, 28, 28, 1]
        net = tf.nn.conv2d(img, self.W1, padding="SAME", strides=[1, 1, 1, 1])
        net = tf.nn.relu(net)
        net = tf.nn.max_pool(net, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
        
        #net -> [batch, 14, 14, 32]
        net = tf.nn.conv2d(net, self.W2, padding="SAME", strides=[1, 1, 1, 1])
        net = tf.nn.relu(net)
        net = tf.nn.max_pool(net, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
        
        net = tf.nn.conv2d(net, self.W3, padding="SAME", strides=[1, 1, 1, 1])
        net = tf.nn.relu(net)
        
        net = tf.nn.conv2d(net, self.W4, padding="SAME", strides=[1, 1, 1, 1])
        net = tf.nn.relu(net)
        
        net = tf.nn.conv2d(net, self.W5, padding="SAME", strides=[1, 1, 1, 1])
        net = tf.nn.relu(net)
        
        net = tf.nn.conv2d(net, self.W6, padding="SAME", strides=[1, 1, 1, 1])
        net = tf.nn.relu(net)
        
        #net -> [batch, 7, 7, 64]
        net = tf.reshape(net, [-1, 7*7*64])
        net = tf.matmul(net, self.W7)
        net = tf.nn.relu(net)
        
        #net -> [batch, 32]
        net = tf.matmul(net, self.W8)
        net = tf.add(net, self.b8)
        
        #net -> [batch, 6]
        theta = net
        return theta
    
        
    def Grid_Generator(self, theta):
        theta = tf.reshape(theta, shape=[-1, 2, 3])
        
        width = self.output_size
        height = self.output_size
        
        #[-1.0 ~ 1.0] normalize
        x = tf.linspace(-1.0, 1.0, width)
        y = tf.linspace(-1.0, 1.0, height)
        
        #make xy grid(meshgrid)
        x_t, y_t = tf.meshgrid(x, y)
        
        x_t_flat = tf.reshape(x_t, [-1])
        y_t_flat = tf.reshape(y_t, [-1])
        ones     = tf.ones_like(y_t_flat)
        
        #make grid with ones
        grid = tf.stack([x_t_flat, y_t_flat, ones])
        
        #make batch size
        grid = tf.expand_dims(grid, axis=0)
        batch_grid = tf.tile(grid, tf.stack([self.batch_size, 1, 1]))
        
        theta = tf.cast(theta, 'float32')
        batch_grid = tf.cast(batch_grid, 'float32')
        
        #theta x batch_grid
        
        batch_grid = tf.matmul(theta, batch_grid)
        
        #reshape
        #print(np.shape(batch_grid))
        batch_grid = tf.reshape(batch_grid, [self.batch_size, 2, width, height])
        
        return batch_grid
    
    def Get_Pixel_Value(self, img, x, y):
        shape = np.shape(x)
        batch_size = shape[0]
        height = shape[1]
        width = shape[2]

        batch_idx = tf.range(0, batch_size)
        batch_idx = tf.reshape(batch_idx, (batch_size, 1, 1))
        b = tf.tile(batch_idx, (1, height, width))

        indices = tf.stack([b, y, x], 3)

        return tf.gather_nd(img, indices)
        
    def Bilinear_Sampler(self, img, grid):
        x = grid[:, 0, :, :]
        y = grid[:, 1, :, :]
        
        H = tf.shape(img)[1]
        W = tf.shape(img)[2]
        
        max_x = tf.cast(W-1, 'int32')
        max_y = tf.cast(H-1, 'int32')
        zero = tf.zeros([], dtype = 'int32')
        
        x = tf.cast(x, 'float32')
        y = tf.cast(y, 'float32')
        
        x = 0.5 * ((x + 1.0) * tf.cast(max_x-1, 'float32'))
        y = 0.5 * ((y + 1.0) * tf.cast(max_y-1, 'float32'))
        
        #1 이하 값 없애기
        x0 = tf.cast(tf.floor(x), 'int32')
        x1 = x0 + 1
        y0 = tf.cast(tf.floor(y), 'int32')
        y1 = y0 + 1
        
        x0 = tf.clip_by_value(x0, zero, max_x)
        x1 = tf.clip_by_value(x1, zero, max_x)
        y0 = tf.clip_by_value(y0, zero, max_y)
        y1 = tf.clip_by_value(y1, zero, max_y)
        
        # get pixel value at corner coords
        Ia = self.Get_Pixel_Value(img, x0, y0)
        Ib = self.Get_Pixel_Value(img, x0, y1)
        Ic = self.Get_Pixel_Value(img, x1, y0)
        Id = self.Get_Pixel_Value(img, x1, y1)

        # recast as float for delta calculation
        x0 = tf.cast(x0, 'float32')
        x1 = tf.cast(x1, 'float32')
        y0 = tf.cast(y0, 'float32')
        y1 = tf.cast(y1, 'float32')

        # calculate deltas
        wa = (x1-x) * (y1-y)
        wb = (x1-x) * (y-y0)
        wc = (x-x0) * (y1-y)
        wd = (x-x0) * (y-y0)

        # add dimension for addition
        wa = tf.expand_dims(wa, axis=3)
        wb = tf.expand_dims(wb, axis=3)
        wc = tf.expand_dims(wc, axis=3)
        wd = tf.expand_dims(wd, axis=3)

        # compute output
        out = tf.add_n([wa*Ia, wb*Ib, wc*Ic, wd*Id])

        return out
        
        
    def load_data(self, rot = False, trans = False):
        """image, _ = mnist.train.next_batch(self.batch_size)#batch_xs, _ : image, label
        image = np.reshape(image, [-1, 28, 28])
        
        angle = np.random.randint(0, 356, self.batch_size)
        rotated_image = []
        for i in range(self.batch_size):
            tmp = ndimage.rotate(image[i], angle[i], reshape=False)
            #tmp = cv2.resize(tmp, (40, 40))
            rotated_image.append(tmp)
        label_image = []
        for i in range(self.batch_size):
            #tmp = cv2.resize(image[i], (40, 40))
            #label_image.append(tmp)
            label_image.append(image[i])
            
        rotated_image = np.expand_dims(rotated_image, axis=3)    
        label_image = np.expand_dims(label_image, axis=3)"""
        
        image, _ = mnist.train.next_batch(self.batch_size)#batch_xs, _ : image, label
        image = np.reshape(image, [-1, 28, 28])

        BLACK = [0, 0, 0]
        translated_image = []

        for i in range(len(image)):
            array = np.random.randint(-30, 30, size=(2), dtype=np.int8)
            translate_matrix = np.float32([[1, 0, array[0]],[0, 1, array[1]]])
            constant = cv2.copyMakeBorder(image[i], 30, 30, 30, 30, cv2.BORDER_CONSTANT, value=BLACK)
            img_translation = cv2.warpAffine(constant, translate_matrix, (88, 88))
            img_translation = cv2.resize(img_translation, (28, 28))
            translated_image.append(img_translation)
            
        label_image = []
        for i in range(self.batch_size):
            label_image.append(image[i])
        translated_image = np.expand_dims(translated_image, axis = 3)
        label_image = np.expand_dims(label_image, axis=3)
        
        #return label_image, rotated_image, angle
        return label_image, translated_image
        
    def spatial_transformer(self, _input):
        theta = self.Localisation_Net(_input)
        grid = self.Grid_Generator(theta)
        output = self.Bilinear_Sampler(_input, grid)
        
        return output
    
    
    def model(self):
        self.stn_output = self.spatial_transformer(self.X)
        self.l2_loss = tf.nn.l2_loss((self.stn_output - self.Y) / self.batch_size)
        self.loss = tf.reduce_mean(self.l2_loss)
        
        global_step = tf.Variable(0, trainable=False)
        starter_learning_rate = 1e-06
        lr = tf.train.exponential_decay(starter_learning_rate, global_step,100000, 0.333, staircase=True)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = lr).minimize(self.loss, global_step = global_step)
        
    def train(self):
        data_size = 60000
        batch_size = self.batch_size
        total_batch = data_size//batch_size

        loss_data = []
        
        SAVE_PATH = "C:/Users/SimJaekyu/Documents/Jupyter Notebook/Spatial_Transformer_Net/Weight/Weight.ckpt"
        print("session start")
        self.sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()
        try:
            saver.restore(self.sess, SAVE_PATH)
            print("Existing Weight.ckpt file load")
            print("Training Start")
        except:
            print("No Training Weight.ckpt exist")
            print("Make New Weight.ckpt file for training")
            print("Training Start")
        
        loss_data = []
        for epoch in range(self.epochs):#15
            print("epoch",epoch+1, "start")
            for i in range(total_batch):#total_batch
                #data load, batch 생성
                label_img, translated_img = self.load_data()
                #print("self.X's shape : ", np.shape(rotate_img), "self.Y's shape : ", np.shape(label_img))
                total_loss_opt, net_loss, predict = self.sess.run([self.optimizer, self.l2_loss, self.stn_output], feed_dict = {self.X : translated_img, self.Y : label_img})
                loss_data.append(net_loss)

            print(net_loss)
            plt.plot(loss_data)
            plt.show()
            
            test_index = 0
            print("--------------------predict--------------------")
            predict = np.squeeze(predict)
            plt.imshow(predict[test_index])
            plt.show()
            print("------------------rotate_img-------------------")
            translated_img = np.squeeze(translated_img)
            plt.imshow(translated_img[test_index])
            plt.show()
            print("------------------label_img-------------------")
            label_img = np.squeeze(label_img)
            plt.imshow(label_img[test_index])
            plt.show()
            print("epoch end")
            
            #print("loss : ", Vector_loss)
            #print('\n')
            
            
            saver.save(self.sess, SAVE_PATH)
            
            #return rotate_img, label_img
print("cell end")

cell end


In [ ]:
#roatation train
with tf.Session() as sess:
    obj = STN(sess = sess, batch_size=64, epochs=400)
    obj.train()

In [ ]:
#translation train
with tf.Session() as sess:
    obj = STN(sess = sess, batch_size=64, epochs=400)
    obj.train()